# Практика
Используемые библиотеки

In [1]:
import pandas as pd
import numpy as np
import requests
import io
import re

## Загрузка DataFrame
### Задача 1
На основании данных портала "Открытые данные России" о результатах Химического анализа родника в Нескучном саду https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad
средствами библиотеки __Pandas__ сформируйте поле выводов по каждому анализирумомому параметру.
Например, по показателю _pH_ получен результат _8.4 единицы pH_ при нормативе от _6 до 9 единиц pH_. Т.о. по данному показателю результат анализа в норме.
Для решения задачи необходимо программно "прочитать и понять" значение столбца "Норматив" и выделенное численное значение сравнить с нормативом согласно логике норматива. Например, __6 >= pH >= 9__.
В итоговом DataFrame столбец "Показатель" сделайте индексным.


Загзрузка DataFrame выполняется непосредственно c сайта "Открытые данные России" https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad/data-20160608T1215-structure-20160608T1215.csv (см. код ниже).


In [2]:
url ="https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad/data-20160608T1215-structure-20160608T1215.csv"
s = requests.get(url).content
df=pd.read_csv(io.StringIO(s.decode('UTF8')))
display(df)

,Показатель,Единица измерений,Результат анализа,Норматив
0,pH,единицы pH,8.4,в пределах 6-9
1,Запах,баллы,1,не более 2-3
2,Цветность,градусы,б/цвета,не более 30
3,Жёсткость,мг-эквл/дм3,9.199999999999999,в пределах 7-10
4,Аммиак и аммоний-ион (по азоту),мг/дм3,0.42,"не более 1,5"
5,Нитриты (по NO2),мг/дм3,0.017,"не более 3,3"
6,Нитраты (по NO3),мг/дм3,24,не более 45
7,Фосфаты (P),мг/дм3,0.36,"не более 3,5"
8,Хлориды (Cl),мг/дм3,200,не более 350
9,Сульфаты (SO4),мг/дм3,189.5,не более 500


In [3]:
# Ваше решение
derived_df = df
derived_df.set_index("Показатель")

def parse_norms_column(text: str) -> str:
    closed_range_pattern = r'в пределах'
    open_range_pattern =  r'не более'
    number_patter = r'\d+\-?\,?\d*'
    
    if re.match(closed_range_pattern, text):
        substring = re.search(number_patter, text).group()
        seq = substring.split("-")
        return (float(seq[0]), float(seq[1]))

    elif re.match(open_range_pattern, text):
        substring = re.search(number_patter, text).group()
        seq = substring.split("-")
        if len(seq) == 1:
            return (0.0, float(seq[0].replace(",", ".")))
        elif len(seq) == 2:
            return (0.0, float(seq[1].replace(",", ".")))
        else:
            raise Exception
    
parse_norms_column("не более 1,5")
parse_norms_column("в пределах 6-9")
parse_norms_column("не более 2-3")
parse_norms_column("в пределах 7-10")

(7.0, 10.0)

In [4]:
derived_df["Заключение"] = [""] * len(derived_df)
for row in derived_df.index:
    raw_value = derived_df["Норматив"].loc[row]
    value = parse_norms_column(raw_value)
    
    raw_result = derived_df["Результат анализа"].loc[row]
    try:
        raw_result = float(raw_result)
    except:
        raw_result = 0
    if value[0] <= raw_result <= value[1]:
        derived_df["Заключение"].loc[row] = "Норма"
    else:
        derived_df["Заключение"].loc[row] = "Отклонение от нормы"
        
derived_df

,Показатель,Единица измерений,Результат анализа,Норматив,Заключение
0,pH,единицы pH,8.4,в пределах 6-9,Норма
1,Запах,баллы,1,не более 2-3,Норма
2,Цветность,градусы,б/цвета,не более 30,Норма
3,Жёсткость,мг-эквл/дм3,9.199999999999999,в пределах 7-10,Норма
4,Аммиак и аммоний-ион (по азоту),мг/дм3,0.42,"не более 1,5",Норма
5,Нитриты (по NO2),мг/дм3,0.017,"не более 3,3",Норма
6,Нитраты (по NO3),мг/дм3,24,не более 45,Норма
7,Фосфаты (P),мг/дм3,0.36,"не более 3,5",Норма
8,Хлориды (Cl),мг/дм3,200,не более 350,Норма
9,Сульфаты (SO4),мг/дм3,189.5,не более 500,Норма


## Теория вероятности. События

Требуется сгенерировать необходимые выборки и произвести по ним расчеты

### Задача 2
В ящике 5 апельсинов и 4 яблока. Наудачу выбираются 3 фрукта. Какова вероятность, что все три фрукта – апельсины?

В интернете полученный аналитически ответ 0.119. Подтверждается ли он эксперементально?


In [14]:
# Ваше решение
bucket = (["orange"] * 5) + (["apple"] * 4)
np.random.shuffle(bucket)
N = 2000
positive_tests = 0 

for _ in range(0, N):
    case = np.random.choice(bucket, 3, replace=False)
    if np.array_equal(case, ["orange"] * 3):
        positive_tests += 1

propability = positive_tests / N
print(f"Вероятность выбрать подряд три апельсина: {propability}")

Вероятность выбрать подряд три апельсина: 0.1155


### Вывод:
В целом результат подтверждается, есть небольшое отклонение около 0.7%

### Задача 3
Мастер, имея 10 деталей, из которых 3 – нестандартных, проверяет детали одну за другой, пока ему не попадется стандартная. Какова вероятность, что он проверит ровно две детали?


В интернете полученный аналитически ответ 7/30 или 0.23333. Подтверждается ли он эксперементально?

In [39]:
# Ваше решение
details = ["standard"] * 7 + ["nonstandard"] * 3
np.random.shuffle(details)
N = 2000

positive_tests = 0
for _ in range(0, N):
    case = np.random.choice(details, 2, replace=False)
    if np.array_equal(case, ["nonstandard", "standard"]):
        positive_tests += 1
        
propability = positive_tests / N
print(f"Вероятность проверить ровне две детали, aka проверить двенестандартных подряд: {propability}")

Вероятность проверить ровне две детали, aka проверить двенестандартных подряд: 0.2365


### Вывод:
В целом результат подтверждается, есть небольшое отклонение также около 0.7%
Думаю можно увеличить N до 10_000